In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [24]:
df_train=pd.read_csv('CarInsurance_train.csv')

In [25]:
df_train.head(5)

,Id,Age,Job,Marital,Education,Default,Balance,HHInsurance,CarLoan,Communication,LastContactDay,LastContactMonth,NoOfContacts,DaysPassed,PrevAttempts,Outcome,CallStart,CallEnd,CarInsurance
0,1,32,management,single,tertiary,0,1218,1,0,telephone,28,jan,2,-1,0,NaN,13:45:20,13:46:30,0
1,2,32,blue-collar,married,primary,0,1156,1,0,NaN,26,may,5,-1,0,NaN,14:49:03,14:52:08,0
2,3,29,management,single,tertiary,0,637,1,0,cellular,3,jun,1,119,1,failure,16:30:24,16:36:04,1
3,4,25,student,single,primary,0,373,1,0,cellular,11,may,2,-1,0,NaN,12:06:43,12:20:22,1
4,5,30,management,married,tertiary,0,2694,0,0,cellular,3,jun,1,-1,0,NaN,14:35:44,14:38:56,0


In [26]:
df_train.isnull().sum()

,0
Id,0
Age,0
Job,19
Marital,0
Education,169
Default,0
Balance,0
HHInsurance,0
CarLoan,0
Communication,902


In [27]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Id                4000 non-null   int64 
 1   Age               4000 non-null   int64 
 2   Job               3981 non-null   object
 3   Marital           4000 non-null   object
 4   Education         3831 non-null   object
 5   Default           4000 non-null   int64 
 6   Balance           4000 non-null   int64 
 7   HHInsurance       4000 non-null   int64 
 8   CarLoan           4000 non-null   int64 
 9   Communication     3098 non-null   object
 10  LastContactDay    4000 non-null   int64 
 11  LastContactMonth  4000 non-null   object
 12  NoOfContacts      4000 non-null   int64 
 13  DaysPassed        4000 non-null   int64 
 14  PrevAttempts      4000 non-null   int64 
 15  Outcome           958 non-null    object
 16  CallStart         4000 non-null   object
 17  CallEnd       

In [28]:
categorical_columns = ['Job', 'Education', 'Communication', 'Outcome']
df_train[categorical_columns] = df_train[categorical_columns].fillna('Unknown')
df_train.fillna(df_train.median(numeric_only=True), inplace=True)


In [29]:
print("Data types before encoding:\n", df_train.dtypes)

# Convert categorical columns to dummy variables
df_train = pd.get_dummies(df_train, columns=['Job', 'Marital', 'Education', 'Communication', 'Outcome', 'LastContactMonth'], drop_first=True)

# Drop irrelevant columns
df_train = df_train.drop(columns=['Id', 'CallStart', 'CallEnd'])

# Check again to ensure all features are numeric
print("Data types after encoding:\n", df_train.dtypes)

Data types before encoding:
 Id                   int64
Age                  int64
Job                 object
Marital             object
Education           object
Default              int64
Balance              int64
HHInsurance          int64
CarLoan              int64
Communication       object
LastContactDay       int64
LastContactMonth    object
NoOfContacts         int64
DaysPassed           int64
PrevAttempts         int64
Outcome             object
CallStart           object
CallEnd             object
CarInsurance         int64
dtype: object
Data types after encoding:
 Age                        int64
Default                    int64
Balance                    int64
HHInsurance                int64
CarLoan                    int64
LastContactDay             int64
NoOfContacts               int64
DaysPassed                 int64
PrevAttempts               int64
CarInsurance               int64
Job_admin.                  bool
Job_blue-collar             bool
Job_entrepreneur    

In [31]:
X = df_train.drop(columns=['CarInsurance']).astype(int)
y = df_train['CarInsurance']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train logistic regression model
log_reg = LogisticRegression(max_iter=1000)  # Increased max_iter to ensure convergence
log_reg.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [33]:
y_pred = log_reg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.73

Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.87      0.80       484
           1       0.72      0.50      0.59       316

    accuracy                           0.73       800
   macro avg       0.73      0.69      0.69       800
weighted avg       0.73      0.73      0.72       800



In [34]:
df_test = pd.read_csv('CarInsurance_test.csv')

In [35]:
df_test[categorical_columns] = df_test[categorical_columns].fillna('Unknown')
df_test.fillna(df_test.median(numeric_only=True), inplace=True)

In [36]:
df_test = pd.get_dummies(df_test, columns=['Job', 'Marital', 'Education', 'Communication', 'Outcome', 'LastContactMonth'], drop_first=True)

# Align columns of test data with training data
df_test = df_test.reindex(columns=X.columns, fill_value=0)

In [37]:
predictions = log_reg.predict(df_test)

# Add predictions to test dataset
df_test['CarInsurance'] = predictions

# Save predictions to a new file
df_test.to_csv('CarInsurance_predictions.csv', index=False)
print("Predictions saved to 'CarInsurance_predictions.csv'")

Predictions saved to 'CarInsurance_predictions.csv'


In [38]:
df=pd.read_csv('CarInsurance_predictions.csv')

In [39]:
df.head()

,Age,Default,Balance,HHInsurance,CarLoan,LastContactDay,NoOfContacts,DaysPassed,PrevAttempts,Job_admin.,Job_blue-collar,Job_entrepreneur,Job_housemaid,Job_management,Job_retired,Job_self-employed,Job_services,Job_student,Job_technician,Job_unemployed,Marital_married,Marital_single,Education_primary,Education_secondary,Education_tertiary,Communication_cellular,Communication_telephone,Outcome_failure,Outcome_other,Outcome_success,LastContactMonth_aug,LastContactMonth_dec,LastContactMonth_feb,LastContactMonth_jan,LastContactMonth_jul,LastContactMonth_jun,LastContactMonth_mar,LastContactMonth_may,LastContactMonth_nov,LastContactMonth_oct,LastContactMonth_sep,CarInsurance
0,25,0,1,1,1,12,12,-1,0,True,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,0
1,40,0,0,1,1,24,1,-1,0,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,0
2,44,0,-1313,1,1,15,10,-1,0,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,0
3,27,0,6279,1,0,9,1,-1,0,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,0
4,53,0,7984,1,0,2,1,-1,0,False,False,False,False,False,False,False,False,False,True,False,True,False,False,True,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,0
